In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

# Parameters
J = 12  # Look-back period for ranking (in months)
max_holding_period = 6  # Maximum holding period (in months)

# Fetch data from Yahoo Finance
stocks = [
    'ASIANPAINT.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'EICHERMOT.NS', 'NESTLEIND.NS', 
    'GRASIM.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ITC.NS', 
    'LT.NS', 'M&M.NS', 'RELIANCE.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 
    'TATASTEEL.NS', 'WIPRO.NS', 'APOLLOHOSP.NS', 'DRREDDY.NS', 'TITAN.NS', 
    'SBIN.NS', 'SRTRANSFIN.NS', 'BPCL.NS', 'KOTAKBANK.NS', 'INFY.NS', 
    'BAJFINANCE.NS', 'ADANIENT.NS', 'SUNPHARMA.NS', 'JSWSTEEL.NS', 'HDFCBANK.NS', 
    'TCS.NS', 'ICICIBANK.NS', 'POWERGRID.NS', 'MARUTI.NS', 'INDUSINDBK.NS', 
    'AXISBANK.NS', 'HCLTECH.NS', 'ONGC.NS', 'NTPC.NS', 'COALINDIA.NS', 
    'BHARTIARTL.NS', 'TECHM.NS', 'MINDTREE.NS', 'DIVISLAB.NS', 'ADANIPORTS.NS', 
    'HDFCLIFE.NS', 'SBILIFE.NS', 'ULTRACEMCO.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS'
]
start_date = '2021-07-01'
end_date = '2024-07-01'

# Fetch monthly data
monthly_data = yf.download(stocks, start=start_date, end=end_date, interval='1mo')

# Fetch daily data
daily_data = yf.download(stocks, start=start_date, end=end_date, interval='1d')

# Use adjusted close prices
df_monthly = monthly_data['Adj Close']
df_daily = daily_data['Adj Close']

# Calculate monthly returns
monthly_returns = df_monthly.pct_change()

# Calculate trailing returns for ranking
trailing_returns = monthly_returns.rolling(window=J).apply(lambda x: (x + 1).prod() - 1, raw=True)

# Initialize DataFrame to store trades
trades_data = pd.DataFrame(columns=['Trade Start Date', 'Stock', 'Action', 'Quantity', 'Trade Opening Price', 'Trade Closing Date', 'Trade Closing Price', 
                                    'Profit/Loss', 'Profit/Loss %', 'Max Drawdown', 'Max Drawdown %', 'Max Upside', 'Max Upside %',
                                    'Rcum', 'R Adjusted', 'R Mean', 'Volatility', 'Weight', 'Holding Period'])

# Dictionary to store individual stock trades
stock_trades = {stock: pd.DataFrame(columns=trades_data.columns) for stock in stocks}

# List to track open positions
open_positions = []

# Dictionary to store entry prices
entry_prices = {}

# Initialize separate DataFrames for returns and weights
portfolio_returns = pd.DataFrame(columns=['Date'] + [f'{stock}_Return' for stock in stocks] + ['Portfolio Return'])
portfolio_weights = pd.DataFrame(columns=['Date'] + [f'{stock}_Weight' for stock in stocks])

def calculate_drawdown_and_upside(stock, start_date, end_date, action):
    stock_data = df_daily.loc[start_date:end_date, stock]
    if action == 'Buy-Sell':
        returns = stock_data / stock_data.iloc[0] - 1
        max_drawdown = returns.min()
        max_upside = returns.max()
    else:  # 'Sell-Buy'
        returns = 1 - stock_data / stock_data.iloc[0]
        max_drawdown = returns.min()
        max_upside = returns.max()
    return max_drawdown, max_upside

def calculate_monthly_metrics(stock, start_date, end_date):
    stock_data = monthly_returns.loc[start_date:end_date, stock].dropna()
    rcum = (1 + stock_data).prod() - 1
    r_mean = stock_data.mean()
    volatility = stock_data.std()
    r_adjusted = r_mean / volatility if volatility != 0 else 0
    return rcum, r_adjusted, r_mean, volatility

def normalize_weights(weights):
    long_sum = np.sum([w for w in weights if w > 0])
    short_sum = np.sum([w for w in weights if w < 0])
    return np.array([w / long_sum if w > 0 else w / abs(short_sum) for w in weights])

def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def mean_variance_optimization(returns, cov_matrix, top_decile, bottom_decile):
    n = len(returns)
    args = (cov_matrix,)
    
    def objective(weights):
        return portfolio_variance(weights, cov_matrix)

    def dollar_neutral(weights):
        return np.sum(weights)

    constraints = [
        {'type': 'eq', 'fun': dollar_neutral, 'tol': 1e-12},
        {'type': 'eq', 'fun': lambda x: np.sum(np.abs(x)) - 1}  # Sum of absolute weights = 1
    ]
    
    initial_weights = np.zeros(n)
    initial_weights[returns.index.isin(top_decile)] = 1 / (2 * len(top_decile))
    initial_weights[returns.index.isin(bottom_decile)] = -1 / (2 * len(bottom_decile))
    
    bounds = []
    for stock in returns.index:
        if stock in top_decile:
            bounds.append((0, 1))
        elif stock in bottom_decile:
            bounds.append((-1, 0))
        else:
            bounds.append((0, 0))
    
    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Dictionary to store cumulative returns and weights for open positions
cumulative_data = {stock: {'Return': 0, 'Weight': 0} for stock in stocks}

def should_close_position(stock, start_date, current_date, action):
    stock_data = df_monthly.loc[start_date:current_date, stock]
    momentum = stock_data.pct_change(periods=1).iloc[-1]
    
    if action == 'Buy-Sell':
        return momentum < 0  # Close long position if momentum turns negative
    else:  # 'Sell-Buy'
        return momentum > 0  # Close short position if momentum turns positive

# Backtest the strategy
for t in range(J, len(trailing_returns)):
    current_date = trailing_returns.index[t]
    current_returns = trailing_returns.iloc[t].dropna()

    # Rank stocks based on trailing returns
    ranked_stocks = current_returns.sort_values(ascending=False)

    # Identify winner and loser portfolios
    top_decile = ranked_stocks.head(len(ranked_stocks) // 10).index
    bottom_decile = ranked_stocks.tail(len(ranked_stocks) // 10).index

    # Calculate portfolio weights using mean-variance optimization
    active_stocks = list(set(top_decile) | set(bottom_decile))
    returns = monthly_returns.loc[:current_date, active_stocks].iloc[-1]
    cov_matrix = monthly_returns.loc[:current_date, active_stocks].cov()
    weights = mean_variance_optimization(returns, cov_matrix, top_decile, bottom_decile)
    
    normalized_weights = normalize_weights(weights)    
    print(f"Sum of weights for {current_date}: {np.sum(weights)}")
    weight_dict = dict(zip(active_stocks, normalized_weights))

    # Close out positions based on dynamic holding period
    positions_to_close = []
    for pos in open_positions:
        holding_period = (current_date - pos['Date']).days // 30  # Approximate months
        if holding_period >= max_holding_period or should_close_position(pos['Stock'], pos['Date'], current_date, pos['Action']):
            positions_to_close.append(pos)

    # Update portfolio returns and weights
    returns_row = {'Date': current_date}
    weights_row = {'Date': current_date}
    portfolio_return = 0

    for stock in stocks:
        if stock in [pos['Stock'] for pos in open_positions]:
            pos = next(p for p in open_positions if p['Stock'] == stock)
            weights_row[f'{stock}_Weight'] = pos['Weight']
            
            if stock in [p['Stock'] for p in positions_to_close]:
                start_price = pos['Price']
                end_price = df_monthly.loc[current_date, stock]

                if pos['Action'] == 'Buy-Sell':
                    return_pct = (end_price / start_price - 1) * 100
                else:  # 'Sell-Buy'
                    return_pct = (start_price / end_price - 1) * 100

                returns_row[f'{stock}_Return'] = return_pct
                portfolio_return += return_pct * pos['Weight']

                # Update cumulative data
                cumulative_data[stock]['Return'] = return_pct
                cumulative_data[stock]['Weight'] = pos['Weight']
            else:
                returns_row[f'{stock}_Return'] = 0
        else:
            returns_row[f'{stock}_Return'] = 0
            weights_row[f'{stock}_Weight'] = 0

    returns_row['Portfolio Return'] = portfolio_return
    portfolio_returns = portfolio_returns.append(returns_row, ignore_index=True)
    portfolio_weights = portfolio_weights.append(weights_row, ignore_index=True)

    # Now actually close the positions and update trades_data
    for pos in positions_to_close:
        # Calculate P&L for closing the position
        if pos['Action'] == 'Buy-Sell':
            p_and_l = (df_monthly.loc[current_date, pos['Stock']] - pos['Price']) * pos['Quantity']
            p_and_l_percent = (df_monthly.loc[current_date, pos['Stock']] / pos['Price'] - 1) * 100
        else:
            p_and_l = (pos['Price'] - df_monthly.loc[current_date, pos['Stock']]) * pos['Quantity']
            p_and_l_percent = (pos['Price'] / df_monthly.loc[current_date, pos['Stock']] - 1) * 100

        # Calculate max drawdown and upside
        max_drawdown, max_upside = calculate_drawdown_and_upside(pos['Stock'], pos['Date'], current_date, pos['Action'])

        # Calculate monthly metrics
        rcum, r_adjusted, r_mean, volatility = calculate_monthly_metrics(pos['Stock'], pos['Date'], current_date)

        # Update the trade in trades_data and stock_trades
        trade_info = {
            'Trade Closing Date': current_date,
            'Trade Closing Price': df_monthly.loc[current_date, pos['Stock']],
            'Profit/Loss': p_and_l,
            'Profit/Loss %': p_and_l_percent,
            'Max Drawdown': max_drawdown,
            'Max Drawdown %': max_drawdown * 100,
            'Max Upside': max_upside,
            'Max Upside %': max_upside * 100,
            'Rcum': rcum,
            'R Adjusted': r_adjusted,
            'R Mean': r_mean,
            'Volatility': volatility,
            'Holding Period': (current_date - pos['Date']).days // 30
        }
        trades_data.loc[pos['TradeIndex'], trade_info] = pd.Series(trade_info)
        stock_trades[pos['Stock']].loc[pos['TradeIndex'], trade_info] = pd.Series(trade_info)

        # Remove the closed position from open_positions and entry_prices
        open_positions.remove(pos)
        entry_prices.pop(pos['Stock'], None)

    # Execute trades for the current month
    for stock in active_stocks:
        if stock not in [pos['Stock'] for pos in open_positions]:
            weight = weight_dict.get(stock, 0)
            action = 'Buy-Sell' if weight > 0 else 'Sell-Buy'
            trade_info = {
                'Trade Start Date': current_date,
                'Stock': stock,
                'Action': action,
                'Quantity': 1,  # Example quantity
                'Trade Opening Price': df_monthly.loc[current_date, stock],
                'Weight': weight
            }
            trades_data = trades_data.append(trade_info, ignore_index=True)
            stock_trades[stock] = stock_trades[stock].append(trade_info, ignore_index=True)
            open_positions.append({
                'Date': current_date,
                'Stock': stock,
                'Action': action,
                'Price': df_monthly.loc[current_date, stock],
                'Quantity': 1,
                'TradeIndex': len(trades_data) - 1,
                'Weight': weight
            })
            entry_prices[stock] = df_monthly.loc[current_date, stock]

# Save results to CSV
trades_data.to_csv('Trades.csv', index=False)
for stock in stocks:
    stock_trades[stock].to_csv(f'{stock}_Trades.csv', index=False)
portfolio_returns.to_csv('Portfolio_Returns.csv', index=False)
portfolio_weights.to_csv('Portfolio_Weights.csv', index=False)

print("Backtest completed and results saved")

[*********************100%***********************]  50 of 50 completed

ERROR 
2 Failed downloads:
ERROR ['MINDTREE.NS']: Exception('MINDTREE.NS: No timezone found, symbol may be delisted')
ERROR ['SRTRANSFIN.NS']: Exception('SRTRANSFIN.NS: No timezone found, symbol may be delisted')



[*********************100%***********************]  50 of 50 completed

ERROR 
2 Failed downloads:
ERROR ['MINDTREE.NS']: Exception('MINDTREE.NS: No timezone found, symbol may be delisted')
ERROR ['SRTRANSFIN.NS']: Exception('SRTRANSFIN.NS: No timezone found, symbol may be delisted')



Sum of weights for 2022-07-01 00:00:00: 1.3877787807814457e-17
Sum of weights for 2022-08-01 00:00:00: 2.7755575615628914e-17
Sum of weights for 2022-09-01 00:00:00: 4.163336342344337e-17
Sum of weights for 2022-10-01 00:00:00: 1.1102230246251565e-16
Sum of weights for 2022-11-01 00:00:00: 0.0
Sum of weights for 2022-12-01 00:00:00: -2.42861286636753e-17
Sum of weights for 2023-01-01 00:00:00: 0.0
Sum of weights for 2023-02-01 00:00:00: -4.163336342344337e-17
Sum of weights for 2023-03-01 00:00:00: 0.0
Sum of weights for 2023-04-01 00:00:00: 5.551115123125783e-17
Sum of weights for 2023-05-01 00:00:00: -2.7755575615628914e-17
Sum of weights for 2023-06-01 00:00:00: 5.551115123125783e-17
Sum of weights for 2023-07-01 00:00:00: 1.1102230246251565e-16
Sum of weights for 2023-08-01 00:00:00: -1.1102230246251565e-16
Sum of weights for 2023-09-01 00:00:00: 2.7755575615628914e-17
Sum of weights for 2023-10-01 00:00:00: -5.551115123125783e-17
Sum of weights for 2023-11-01 00:00:00: 0.0
Sum of